In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import sys
import tqdm
import pathlib
from pathlib import Path
import io
import contextlib
import gaps_online as go
import go_pybindings as gop
import re
from glob import glob
import os

In [ ]:
plt.style.use('publication.rc')

In [ ]:
dataset = pathlib.Path(f'/Users/gracetytus/gaps/waveforms')
files = [f for f in sorted(dataset.glob('*.tof.gaps'))]

In [ ]:
paddle_map = {}
with open(f'/Users/gracetytus/gaps/channel_mapping.csv') as in_file:
    variables = next(in_file).strip().split(',')
    next(in_file)
    for line in in_file:
        row = line.strip().split(',')
        paddle_id = int(row[0])
        paddle_map[paddle_id] = {'a':{'rb':0,'ch':0},'b':{'rb':0,'ch':0}}
        rb, ch = [int(d) for d in row[9].split('-')]
        paddle_map[paddle_id]['a']['rb'] = rb
        paddle_map[paddle_id]['a']['ch'] = ch - 1

        row = next(in_file).strip().split(',')
        rb, ch = [int(d) for d in row[9].split('-')]
        paddle_map[paddle_id]['b']['rb'] = rb
        paddle_map[paddle_id]['b']['ch'] = ch - 1

In [ ]:
pattern = re.compile(r'RB(\d+)_\d{6}_\d{6}UTC\.cali\.tof\.gaps')
calibrations = glob('/Users/gracetytus/gaps/waveforms/calibrations/240808_220952UTC/*.cali.tof.gaps')

calib = {}

for fname in calibrations:
    match = pattern.search(fname)
    if match:
        rbid = match.group(1)
        cali = gop.events.RBCalibration()
        cali.from_file(fname)  # Modify the instance
        calib[int(rbid)] = cali      # Store the modified instance
    else:
        print("No match found for:", fname)

print(calib)

In [ ]:
waveform_a         = []
waveform_b         = []
time_a             = []
time_b             = []
q_a                = []
q_b                = []
v_a                = []
v_b                = []

with contextlib.redirect_stderr(io.StringIO()):
    #for f in tqdm.tqdm(files, desc="Processing files", unit="file", file=sys.stdout):
    for f in files:
        reader = go.rust_api.io.TofPacketReader(str(f), filter=go.rust_api.io.PacketType.TofEvent)
        settings = go.liftof.LiftofSettings()
        settings = settings.from_file('/Users/gracetytus/gaps/waveforms/run134.toml')

        n_packets = 0
        for pack in reader:
            n_packets += 1

        reader.rewind()
    
        for pack in tqdm.tqdm(reader, total=n_packets, file=sys.stdout, position=0):
            ev = go.rust_api.events.TofEvent()
            new_ev = go.liftof.waveform_analysis(ev, settings)

            try:
                ev.from_tofpacket(pack)
                
            except Exception as e:
                print(f"Error at hit {x}: {e}")
                pass
                continue

            for x in range(len(new_ev.hits)):
                try: 
                    paddle = int(new_ev.hits[x].paddle_id)

                    if new_ev.hits[x].charge_a == 0 or new_ev.hits[x].charge_b == 0:
                        continue

                    q = new_ev.hits[x].charge_a
                    v = new_ev.hits[x].peak_a
                    t = new_ev.hits[x].time_a

                    rb = paddle_map[paddle]['a']['rb']
                    ch = paddle_map[paddle]['a']['ch']
                    if ch == 8: continue
                    for waveform in new_ev.waveforms:
                        if waveform.rb_id == rb and waveform.rb_channel == ch:
                            waveform.calibrate(calib[rb])
                            waveform.apply_spike_filter()
                            waveform_a.append(waveform.voltages)
                            time_a.append(t)
                            q_a.append(q)
                            v_a.append(v)
                            break
                    
                    q = new_ev.hits[x].charge_b
                    v = new_ev.hits[x].peak_b
                    t = new_ev.hits[x].time_b
                    
                    rb = paddle_map[paddle]['b']['rb']
                    ch = paddle_map[paddle]['b']['ch']
                    if ch == 8: continue
                    for waveform in new_ev.waveforms:
                        if waveform.rb_id == rb and waveform.rb_channel == ch:
                            waveform.calibrate(calib[rb])
                            waveform.apply_spike_filter()
                            waveform_b.append(waveform.voltages)
                            time_b.append(t)
                            q_b.append(q)
                            v_b.append(v)
                            break
                except Exception as e:
                    print(f"Error at hit {x}: {e}")
                    continue
                    
waveform_a = np.array(waveform_a)
waveform_b = np.array(waveform_b)
time_a = np.array(time_a)
time_b = np.array(time_b)
q_a = np.array(q_a)
q_b = np.array(q_b)
v_a = np.array(v_a)
v_b = np.array(v_b)

In [ ]:
for in range(100):
    plt.plot(waveform_a[x])
    plt.ylim(0, 200)
    plt.show()

In [ ]:
fig, (ax_3, ax_6) = plt.subplots(1,2, figsize = (14,7))

x = np.arange(0,len(waveform_a[0]), 1)

for wf, ratio in zip(waveform_a, peak_over_charge_a):
    if ratio < 3.2 and np.all(wf > -200):
        ax_3.plot(x-np.argmax(wf), wf, lw=0.5, alpha=0.25, color = 'navy')
        ax_3.set_title('Waveforms with p/c ratio ~3')
        ax_3.set_xlabel('bins')
        ax_3.set_ylabel('voltage [mV]')
        ax_3.minorticks_on()
    elif ratio > 5.8 and np.all(wf > -200):
        ax_6.plot(x-np.argmax(wf), wf, lw=0.5, alpha=0.25, color = 'navy')
        ax_6.set_title('Waveforms with p/c ratio ~6')
        ax_6.set_xlabel('bins')
        ax_6.set_ylabel('voltage [mV]')
        ax_6.minorticks_on()
plt.savefig('/Users/gracetytus/Downloads/composite_clean.pdf')

In [ ]:

mean_wf = np.mean(waveform_a[peak_over_charge_a < 3.2], axis=0)

In [ ]:
mean_wf = np.mean(waveform_a[peak_over_charge_a < 3.2], axis=0)
plt.plot(mean_wf[1:], label = r'$\frac{p}{c} \approx 3$')
mean_wf = np.mean(waveform_a[peak_over_charge_a > 5.8], axis=0)
plt.plot(mean_wf[1:], label = r'$\frac{p}{c} \approx 6$')
plt.xlabel('bins')
plt.ylabel('voltage [mV]')
plt.legend()
plt.title('Avg. waveforms')
plt.savefig('/Users/gracetytus/Downloads/avgwf.png')

In [ ]:
plt.plot(mean_wf)
plt.title('avg. waveform in region p/c ~3')
plt.xlabel('bins')
plt.ylabel('voltage [mV]')
plt.ylim
plt.savefig('/Users/gracetytus/Downloads/avg3.png')

In [ ]:
mean_wf = np.mean(waveform_a[peak_over_charge_a > 5.8], axis=0)

In [ ]:
plt.plot(mean_wf)
plt.title('avg. waveform in region p/c ~6')
plt.xlabel('bins')
plt.ylim(0, 30)
plt.ylabel('voltage [mV]')
plt.savefig('/Users/gracetytus/Downloads/avg6.png')

In [ ]:
peak_over_charge_a = np.array(peak_over_charge_a)

In [ ]:
(ratio > 2.8 and ratio < 3.2) or (ratio > 4.4 and ratio < 4.6) or (ratio > 5.8 and ratio < 6.2)

In [ ]:
_ = plt.hist(peak_over_charge_a, bins='auto')

In [ ]:
_ = plt.hist(peak_over_charge_a[np.all(waveform_a>-200, axis=1)], bins='auto')

In [ ]:
diff = time_a - time_b
plt.hist(diff)

In [ ]:
t_diff = []
ratio = []
for x in range(len(time_a)):
    if q_a[x] == 0: continue
    t = time_a[x] - time_b[x]
    r = v_a[x] / q_a[x] 
    t_diff.append(t)
    ratio.append(r)
    plt.plot(waveform_a[x])
    plt.show()

In [ ]:
t_diff = np.abs(t_diff)

plt.hist2d(ratio, t_diff, bins = (200, 200), cmap=plt.cm.gnuplot2)
plt.xlabel('ratio peak to charge')
plt.ylabel('time_a - time_b')
plt.title('comparison with ratio from a side')
plt.ylim(0, 20)
plt.xlim(2, 8)
plt.colorbar()
plt.show()
#plt.savefig('/Users/gracetytus/Downloads/2d_sidea.png')

In [ ]:
_ = plt.hist(peak_over_charge_a[np.any(waveform_a<-200, axis=1)], bins='auto')
plt.title('distribution of data mangled events')
plt.xlabel('peak / charge')
plt.ylabel('n')
plt.savefig('/Users/gracetytus/Downloads/mangled_dist.png')

In [ ]:
len(waveform_a)

In [ ]:
waveform_a>-200

In [ ]:
np.sum(np.any(waveform_a<-200, axis=1))

In [ ]:
total = 0
for wf in waveform_a:
    if np.any(wf<-200):
        total += 1

In [ ]:
total

In [ ]:
for wf in waveform_a[np.all(waveform_a>-200, axis=1)]:
    plt.plot(wf)

In [ ]:
t_diff = []
ratio = []
for x in range(len(time_a)):
    if q_b[x] == 0: continue
    t = time_b[x] - time_a[x]
    r = v_b[x] / q_b[x] 
    t_diff.append(t)
    ratio.append(r)
t_diff = np.abs(t_diff)

plt.hist2d(ratio, t_diff, bins = (200, 200), cmap=plt.cm.gnuplot2)
plt.xlabel('ratio peak to charge')
plt.ylabel('time_b - time_a')
plt.title('comparison with ratio from b side')
plt.ylim(0, 20)
plt.xlim(2, 8)
plt.colorbar()
plt.show()
#plt.savefig('/Users/gracetytus/Downloads/2d_sideb.png')

In [ ]:
time_a[0]

In [ ]:
time_b[0]

In [ ]:
time_a[0] - time_b[0]